In [1]:
import cv2
import numpy as np
import time # 출력 속도 조절을 위해 임포트

In [2]:
# --- 1. 초기 설정 및 비디오 읽기 ---
# NOTE: 이 코드를 실행하기 전에 반드시 Google Drive를 마운트하고
# 아래 경로가 정확한지 확인해야 합니다.
video_path = 'newyork.mp4'
cap = cv2.VideoCapture(video_path) # 비디오 캡처 객체 생성

# 비디오 파일이 제대로 열렸는지 확인
if not cap.isOpened():
    print(f"오류: 비디오 파일 '{video_path}'을 열 수 없습니다.")
    print("Google Drive가 마운트되었고 경로가 정확한지 확인해 주세요.")
    exit()



In [ ]:
# 2. 첫 프레임에서 특징점 찾기
ret, old_frame = cap.read() # 첫 프레임 읽기

# 프레임을 제대로 읽었는지(ret=True) 한 번 더 확인: 이 부분이 기존 오류의 원인일 가능성이 높습니다.
if not ret or old_frame is None:
    print("오류: 첫 번째 프레임을 읽지 못했습니다. 비디오 파일이 손상되었을 수 있습니다.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(old_gray, 
                        maxCorners=100, 
                        qualityLevel=0.3, 
                        minDistance=7)

# 3.Lucas-Kanade
lk_params = dict(
    winSize=(15,15), # 윈도우 사이즈(크기)
    maxLevel=2,      # 피라미드 레벨
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)
# 이미지 피라미드 (coarse to fine strategy)
# 0 : 원본, 1 : 1단계 다운샘플링

# 4.비디오 처리루프, 광학흐름(optical flow) 계산
frame_count = 0
MAX_FRAMES_TO_PROCESS = 100
DISPLAY_EVERY_N_FRAMES = 20

while cap.isOpened() and frame_count < MAX_FRAMES_TO_PROCESS:
    ret, frame = cap.read()
    if not ret:
        break
    
    if not ret or frame is None:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    new_corners, status, error = cv2.calcOpticalFlowPyrLK(
        old_gray, frame_gray, corners, None, **lk_params)

    # 5. 좋은 특징점 추출
    good_new = new_corners[status==1]
    good_old = corners[status==1]

    if track_frame is None:
        track_frame = frame.copy()
    else:
        track_frame = track_frame

    # 6.움직임 그리기
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int) # 현재위치
        c, d = old.ravel().astype(int) # 이전위치

        # 이전과 현재 위치를 연결
        mask = cv2.line(track_frame, (a, b), (c, d), (0, 255, 0), 2)
        frame = cv2.circle(track_frame, (a, b), 5, (0, 0, 255), -1)

    if frame_count % DISPLAY_EVERY_N_FRAMES == 0:
        cv2.imshow('frame', frame)
        time.sleep(1)

    # 업데이트
    old_gray = frame_gray.copy()
    # (포인트 개수, 채널, 좌표값 개수)
    corners = good_new.reshape(-1, 1, 2) # (N, 1, 2)

cap.release()
cv2.destroyAllWindows()

NameError: name 'track_frame' is not defined